# Algorithmic Trading System - Backtest Example

This notebook demonstrates how to run a backtest using the algorithmic trading system.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
from datetime import datetime, timedelta

from src.data.data_fetcher import DataFetcher
from src.strategy.example_strategies import SMAcrossoverStrategy, RSIStrategy
from src.backtesting.backtest_engine import BacktestEngine
from src.backtesting.performance import PerformanceAnalyzer

## 1. Configure Backtest Parameters

In [ ]:
SYMBOLS = ['SPY', 'QQQ']
START_DATE = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')
END_DATE = datetime.now().strftime('%Y-%m-%d')
INITIAL_CAPITAL = 100000

print(f"Backtesting {', '.join(SYMBOLS)} from {START_DATE} to {END_DATE}")

## 2. Fetch Historical Data

In [ ]:
data_fetcher = DataFetcher()

data = {}
for symbol in SYMBOLS:
    print(f"Fetching data for {symbol}...")
    data[symbol] = data_fetcher.fetch_historical_data(symbol, START_DATE, END_DATE)
    print(f"  Loaded {len(data[symbol])} bars")

data['SPY'].head()

## 3. Initialize Strategy and Backtest Engine

In [ ]:
strategy = SMAcrossoverStrategy(short_window=20, long_window=50)

engine = BacktestEngine(
    strategy=strategy,
    initial_capital=INITIAL_CAPITAL,
    commission=0.0,
    slippage=0.001
)

for symbol in SYMBOLS:
    engine.load_data(symbol, data[symbol])

## 4. Run Backtest

In [ ]:
results = engine.run(SYMBOLS)
engine.print_results()

## 5. Visualize Performance

In [ ]:
analyzer = PerformanceAnalyzer(results)

analyzer.plot_equity_curve()
analyzer.plot_drawdown()
analyzer.plot_returns_distribution()

## 6. Analyze Trades

In [ ]:
trades = results['trades']

print(f"Total Trades: {len(trades)}")
print("\nFirst 10 trades:")
for trade in trades[:10]:
    print(f"  {trade.timestamp}: {trade.side} {trade.quantity} {trade.symbol} @ ${trade.price:.2f}")